In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib


In [2]:
# Load and preprocess the data
data = pd.read_csv("D:\\finalProject\\landmarks_all.csv")
X = data.drop(columns=["label"]).values
y = data["label"]

In [3]:
data.head()

,landmark_1,landmark_2,landmark_3,landmark_4,landmark_5,landmark_6,landmark_7,landmark_8,landmark_9,landmark_10,...,landmark_91,landmark_92,landmark_93,landmark_94,landmark_95,landmark_96,landmark_97,landmark_98,landmark_99,label
0,0.474078,0.255586,-0.845679,0.492901,0.215314,-0.802644,0.505039,0.216036,-0.803109,0.515049,...,0.356639,2.313848,0.377244,0.502491,2.394536,-0.002479,0.381498,2.377426,-0.099456,a-abc
1,0.475068,0.255527,-1.028458,0.493540,0.215332,-0.985430,0.505772,0.216051,-0.985767,0.515587,...,0.358301,2.340979,0.728185,0.504685,2.416029,0.110209,0.381616,2.409318,0.237820,a-abc
2,0.475504,0.255363,-1.009970,0.493964,0.215333,-0.965716,0.506334,0.216048,-0.966104,0.516079,...,0.358707,2.358446,0.656555,0.504669,2.430796,0.102040,0.381559,2.424881,0.166357,a-abc
3,0.475864,0.255349,-1.026346,0.494418,0.215733,-0.981857,0.506879,0.216520,-0.982215,0.516564,...,0.362065,2.366050,0.698646,0.508891,2.437438,0.104160,0.384000,2.432841,0.210205,a-abc
4,0.476150,0.255345,-1.036166,0.494780,0.216127,-0.992960,0.507264,0.217017,-0.993278,0.516975,...,0.373359,2.372369,0.749761,0.519158,2.440346,0.085969,0.392524,2.440275,0.263123,a-abc


In [4]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
# Remove classes with fewer than 2 samples
class_counts = np.bincount(y_encoded)
valid_classes = np.where(class_counts > 1)[0]
valid_indices = np.isin(y_encoded, valid_classes)
X_valid = X[valid_indices]
y_valid = y_encoded[valid_indices]

# Stratified split for balanced training and validation sets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, val_index in split.split(X_valid, y_valid):
    X_train, X_val = X_valid[train_index], X_valid[val_index]
    y_train, y_val = y_valid[train_index], y_valid[val_index]

In [7]:
# Reshape input for LSTM if necessary
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [8]:
# Save the label encoder for future use
joblib.dump(label_encoder, r"D:\\finalProject\\training the data\\label_encoder.pkl")

['D:\\\\finalProject\\\\training the data\\\\label_encoder.pkl']

In [10]:
from tensorflow.keras.layers import Bidirectional, Dropout


In [11]:

model = Sequential()
model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

C:\Users\f1930\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
from tensorflow.keras.optimizers import Adam


In [14]:
# Compile with a lower learning rate
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train for more epochs
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64)

Epoch 1/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 66s 28ms/step - accuracy: 0.0018 - loss: 8.8088 - val_accuracy: 0.0064 - val_loss: 7.6844
Epoch 2/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 64s 29ms/step - accuracy: 0.0077 - loss: 7.6098 - val_accuracy: 0.0156 - val_loss: 7.0743
Epoch 3/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 64s 29ms/step - accuracy: 0.0148 - loss: 7.0818 - val_accuracy: 0.0254 - val_loss: 6.6545
Epoch 4/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 66s 30ms/step - accuracy: 0.0211 - loss: 6.7009 - val_accuracy: 0.0376 - val_loss: 6.3188
Epoch 5/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 72s 32ms/step - accuracy: 0.0323 - loss: 6.3831 - val_accuracy: 0.0527 - val_loss: 6.0310
Epoch 6/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 79s 35ms/step - accuracy: 0.0417 - loss: 6.1125 - val_accuracy: 0.0666 - val_loss: 5.7765
Epoch 7/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 84s 38ms/step - accuracy: 0.0515 - loss: 5.8731 - val_accuracy: 0.0807 - val_loss: 5.5503
Epoch 8/50
2241/2241 ━━━━━━━━━━━━━━━━━━━━ 82s 37ms/step - accuracy: 0.0612 -

In [15]:
# Save the model
model.save("D:\\finalProject\\training the data\\asl_sign_model2.h5")

print("Model 2 trained and saved.")

Model 2 trained and saved.
